In [1]:
from nnsight import CONFIG
NDIF_API = ""
HF_TOKEN = ""

CONFIG.set_default_api_key(NDIF_API)
!huggingface-cli login --token {HF_TOKEN}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `main_token` has been saved to /disk/u/gio/.cache/huggingface/stored_tokens
Your token has been saved to /disk/u/gio/.cache/huggingface/token
Login successful.
The current active token is: `main_token`


In [2]:
import torch
import nnsight
from nnsight import LanguageModel

In [3]:
model = LanguageModel("meta-llama/Meta-Llama-3.1-70B")
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 8192)
    (layers): ModuleList(
      (0-79): 80 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=8192, out_features=8192, bias=False)
          (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
          (o_proj): Linear(in_features=8192, out_features=8192, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (up_proj): Linear(in_features=8192, out_features=28672, bias=False)
          (down_proj): Linear(in_features=28672, out_features=8192, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((8192,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((8192,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((8192,), eps=1e-05)
    (rotary_

In [4]:
prompts = [
    "Q: Pick the odd person out: Isaac Newton, Brad Pitt, Leonardo DiCaprio\nA:",
    "Q: Pick the odd person out: Isaac Newton, Albert Einstein, Leonardo DiCaprio\nA:"
]

In [5]:
answers = [
    (" Isaac", (" Brad", " Leonardo")),
    (" Leonardo", (" Isaac", " Albert"))
]

In [6]:
clean_tokens = model.tokenizer(prompts, return_tensors="pt")["input_ids"]
corrupted_tokens = clean_tokens[
    [(i + 1 if i % 2 == 0 else i - 1) for i in range(len(clean_tokens))]
]

In [7]:
#[model.tokenizer.decode(token) for token in clean_tokens]
#[model.tokenizer.decode(token) for token in corrupted_tokens]

In [8]:
all_toks = []
for i in range(len(answers)):
    main_tok = []
    other_toks = []
    for j in range(2):
        if j == 0:
            tok_correct = model.tokenizer(answers[i][j], add_special_tokens=False)['input_ids']
            #print(tok_correct)
            main_tok.extend(tok_correct)
        elif j == 1:
            for k in range(2):
               tok_other = model.tokenizer(answers[i][j][k], add_special_tokens=False)['input_ids']
               #print(tok_other)
               other_toks.extend(tok_other)
    all_toks.append((main_tok, other_toks))
all_toks

[([42608], [17478, 66486]), ([66486], [42608, 17971])]

## Patching Experiment

Now we can run the actual patching intervention.

We have two prompts, a "clean" one and a "corrupted" one.
Intuitively, the model output for each of these prompts should be different: we'd expect the model to asnwer " Isaac" for the clean prompt and " Leonardo" for the corrupted prompt.

In this experiment, we run the model with the clean prompt as an input and then (1) get each layer's output value (i.e., residual stream) and (2) calculate the logit difference between the correct and incorrect answers for this run. Next, we calculate the logit difference between the correct and incorrect answers for the corrupted prompt.

### Step 1: Clean Run

During this clean run, we collect the final output of each layer.
We also record the logit difference in the final model output between the correct answer token " Isaac" and the incorrect tokens " Brad" and " Leonardo".

In [9]:
N_LAYERS = len(model.model.layers)

with model.trace(prompts[0], remote=True) as tracer:
    clean_tokens = tracer.invoker.inputs[0][0]['input_ids'][0]

2025-07-03 12:04:23,655 906834fa-419c-43cd-bff8-a69fcf506e75 - RECEIVED: Your job has been received and is waiting approval.
2025-07-03 12:04:23,829 906834fa-419c-43cd-bff8-a69fcf506e75 - APPROVED: Your job was approved and is waiting to be run.
2025-07-03 12:04:30,713 906834fa-419c-43cd-bff8-a69fcf506e75 - RUNNING: Your job has started running.
2025-07-03 12:04:30,984 906834fa-419c-43cd-bff8-a69fcf506e75 - COMPLETED: Your job has been completed.
